In [3]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import numpy as np
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append('fbsource/fbcode/scripts/psarlin/')
from maploc.utils.geo import Projection, BoundaryBox
from maploc.osm.tiling_v2 import TileManager
from maploc.utils.viz_2d import plot_images, plot_keypoints, add_text, save_plot

sys.path.append('fbsource/fbcode/mapillary/research/metropolis')
from metropolis import Metropolis

In [8]:
data_root = Path('./buckets/metropolis')
met = Metropolis("train", data_root)

In [9]:
is_dummy = np.array([sd_record["ego_pose_token"] == "dummy_ego_pose" for sd_record in met.sample_data])
len(is_dummy), np.mean(is_dummy), np.where(~is_dummy)[0][0]

In [10]:
sides = {"CAM_FRONT", "CAM_RIGHT", "CAM_BACK", "CAM_LEFT"}
# sides = {"CAM_EQUIRECTANGULAR"}

from maploc.mapillary.plane import PlaneModel

def read_pcd(sample_record, pcd_key="MVS"):
    mvs_token = sample_record["data"]["MVS"]
    mvs_record = met.get("sample_data", mvs_token)
    with np.load(met.get_sample_data_path(mvs_token)) as pcd:
        pcd = pcd["points"]
    T_w2pcd = pose_from_sample(mvs_record)
    pcd_w = T_w2pcd.transform_inverse_many(pcd)
    return pcd_w

def fit_plane(pcd_w, t_c2w, num_ransac_iters=100, min_height=1, max_height=3):
    z = pcd_w[:, -1]
    z_cam = t_c2w[-1]
    maybe_ground = (z <= (z_cam - min_height)) & (z >= (z_cam - max_height))
    xyz_subset = pcd_w[maybe_ground]
    if len(xyz_subset) < PlaneModel.min_num_points:
        return None
    plane = PlaneModel(xyz_subset, thresh=0.1, max_trials=num_ransac_iters)
    return plane

from pyquaternion import Quaternion
from opensfm.pygeometry import Pose

def pose_from_sample(data_record):
    cs_record = met.get("calibrated_sensor", data_record["calibrated_sensor_token"])
    pose_record = met.get("ego_pose", data_record["ego_pose_token"])
    q_v2w = pose_record["rotation"]
    t_v2w = pose_record["translation"]
    q_c2v = cs_record["rotation"]
    t_c2v = cs_record["translation"]
    T_w2v = Pose(Quaternion(q_v2w).rotation_matrix, t_v2w).inverse()
    T_v2c = Pose(Quaternion(q_c2v).rotation_matrix, t_c2v).inverse()
    T_w2c = Pose.compose(T_v2c, T_w2v)
    return T_w2c

def process_sample(sample_record):
    tokens = []
    for side in sides:
        cam_token = sample_record["data"].get(side)
        if cam_token is None:
            continue
        cam_record = met.get("sample_data", cam_token)
        if cam_record["ego_pose_token"] == "dummy_ego_pose":
            continue
        tokens.append(cam_token)
    return tokens

In [38]:
scene2sd2cams = {}
for scene in met.scene:
    sd_token = scene["first_sample_token"]
    sd2cams = {}
    while True:
        sample_record = met.get("sample", sd_token)
        ids = process_sample(sample_record)
        if len(ids) > 0:
            sd2cams[sd_token] = ids
        if sd_token == scene["last_sample_token"]:
            break
        sd_token = sample_record["next_sample"]
    if len(sd2cams):
        scene2sd2cams[scene["token"]] = sd2cams

lengths = list(map(len, scene2sd2cams.values()))
plt.hist(lengths);
print(sum(lengths))

In [ ]:
from tqdm import tqdm

sample_tokens = [(sc, sd) for sc, sds in scene2sd2cams.items() for sd in sds]
sample2plane = {}
cam_heights = []  # only for stats

for scene, sd_token in tqdm(sample_tokens):
    sample_record = met.get("sample", sd_token)
    pcd_w = read_pcd(sample_record)
    cam = scene2sd2cams[scene][sd_token][0]
    t_c2w = pose_from_sample(met.get("sample_data", cam)).get_origin()
    plane = fit_plane(pcd_w, t_c2w)
    num_inls = np.sum(plane.inliers)
    angle = np.rad2deg(np.arccos(np.abs(plane.normal[-1])))
    camera_height = t_c2w[-1] - plane.z(*t_c2w[:2])
    if num_inls < 30 or angle > 45 or not (1 <= camera_height <= 3):
        print(f"Maybe wrong fit: #inls={num_inls}, angle={angle:.2f}deg, height={camera_height:.2f}m")
        plane = None
    cam_heights.append(camera_height)
    sample2plane[sd_token] = plane

In [ ]:
cam_heights = np.array(cam_heights)
plt.hist(cam_heights[cam_heights<10])

In [23]:
from opensfm.geo import TopocentricConverter
import json

with open("./data/gps_dict.json") as fd:
   gps_dict = json.load(fd)
gps_dict = {v["orig_fname"]: v for v in gps_dict.values()}

reference = met.geo["reference"]
reference = TopocentricConverter(reference["lat"], reference["lon"], reference["alt"])

def parse_data_record(data_record):
    # data_path = met.get_sample_data_path(data_record["token"])
    filename = data_record["filename"]
    cs_record = met.get("calibrated_sensor", data_record["calibrated_sensor_token"])
    sensor_record = met.get("sensor", cs_record["sensor_token"])

    timestamp = data_record["timestamp"]
    K = np.array(cs_record["camera_intrinsic"])
    w, h = (data_record["width"], data_record["height"])
    camera_dict = {
        "id": cs_record["sensor_token"],
        "model": "PINHOLE",
        "width": w,
        "height": h,
        "params": K[[0, 1, 0, 1], [0, 1, 2, 2]],
    }

    T_w2c = pose_from_sample(data_record)
    return filename, camera_dict, timestamp, T_w2c

from maploc.mapillary.processing import compute_geo_rotation, decompose_rotmat

projection = Projection("EPSG:6498")

In [97]:
all_latlon = []
outputs = {}
for scene, sample2cams in scene2sd2cams.items():
    views = {}
    cameras = {}
    for sample_token, cams in sample2cams.items():
        plane = sample2plane[sample_token]
        if plane is None:
            print(f"Skipping sample {sample_token}")
            continue
        gps = (gps_dict[sample_token]["lat"], gps_dict[sample_token]["lon"])
        for cam_token in cams:
            cam_record = met.get("sample_data", cam_token)
            filename, camera_dict, timestamp, T_w2c = parse_data_record(cam_record)

            xyz_orig = T_w2c.get_origin()
            lla = reference.to_lla(*xyz_orig)
            xy = projection.project(np.array(lla[:2]))
            xyz = np.r_[xy, lla[-1]]
            R_topo2epsg = compute_geo_rotation(reference, projection, xyz_orig)
            R_c2w = R_topo2epsg @ T_w2c.get_R_cam_to_world()
            rpy = decompose_rotmat(R_c2w)
            local_plane_params = np.r_[
                R_topo2epsg @ np.array((plane.a, plane.b, plane.c)),
                plane.d + np.dot(xyz_orig, plane.normal),
            ]
            view = {
                "camera_id": camera_dict["id"],
                "R_c2w": R_c2w,
                "roll_pitch_yaw": rpy,
                "t_c2w": xyz,
                "latlong": lla[:2],
                "height": local_plane_params[-1] / local_plane_params[-2],
                "plane_params": local_plane_params,
                "capture_time": timestamp,
                "gps_position": gps,
            }
            name = filename.rsplit('.', 1)[0]
            views[name] = view
            cameras[camera_dict["id"]] = camera_dict
            all_latlon.append(lla[:2])

    outputs[scene] = {"views": views, "cameras": cameras}

In [99]:
plt.scatter(*projection.project(np.array(all_latlon)).T, s=1)
plt.gca().set_aspect("equal")

In [100]:
from maploc.utils.io import write_json
dump_path = Path("./data/mapillary_dumps_v2/metropolis_train")
dump_path.mkdir(exist_ok=True)
write_json(dump_path / "outputs_per_sequence.json", outputs)

In [101]:
from maploc.mapillary.run import cfg

all_xy = projection.project(np.array(all_latlon))
bbox_total = BoundaryBox(np.min(all_xy, 0), np.max(all_xy, 0))
bbox_tiling = bbox_total + cfg.tiling.margin
print(bbox_tiling)
tile_manager = TileManager.from_bbox(
    dump_path,
    projection,
    bbox_tiling,
    cfg.tiling.tile_size,
    cfg.tiling.ppm,
)
tile_manager.save(dump_path / "tiles.pkl")

In [373]:
!ln -s ./buckets/metropolis $dump_path/images

In [485]:
!mkdir -p $dump_path/images/sample_data/CAM_FRONT
!mkdir -p $dump_path/images/sample_data/CAM_LEFT
!mkdir -p $dump_path/images/sample_data/CAM_RIGHT
!mkdir -p $dump_path/images/sample_data/CAM_BACK
!cp -r ./buckets/metropolis/sample_data/CAM_LEFT/* $dump_path/images/sample_data/CAM_LEFT/
!cp -r ./buckets/metropolis/sample_data/CAM_RIGHT/* $dump_path/images/sample_data/CAM_RIGHT/
!cp -r ./buckets/metropolis/sample_data/CAM_BACK/* $dump_path/images/sample_data/CAM_BACK/
# !manifold getr mapillary_metropolis/tree/sample_data/CAM_FRONT $dump_path/images/sample_data/CAM_FRONT
# !manifold getr mapillary_metropolis/tree/sample_data/CAM_LEFT $dump_path/images2/sample_data/CAM_LEFT
# !manifold getr mapillary_metropolis/tree/sample_data/CAM_RIGHT $dump_path/images2/sample_data/CAM_RIGHT
# !manifold getr mapillary_metropolis/tree/sample_data/CAM_BACK $dump_path/images2/sample_data/CAM_BACK